In [3]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import inspect
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')


In [5]:
# Create an inspector object
inspector = inspect(engine)

# Get a list of all table names
tables = inspector.get_table_names()

# Print the list of tables
print(tables)


['Bookings', 'Facilities', 'Members']


In [6]:
query = "SELECT * FROM Facilities"
df = pd.read_sql_query(query, engine)

# Display the dataframe
print(df.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [7]:
query = "SELECT * FROM Bookings"
df_Bookings = pd.read_sql_query(query, engine)

# Display the dataframe
print(df_Bookings.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [8]:
query = "SELECT * FROM Members"
df_members = pd.read_sql_query(query, engine)

# Display the dataframe
print(df_members.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


In [10]:

# Define the query
query = """
SELECT f.name AS facility_name, 
       SUM(CASE 
           WHEN b.memid = 0 THEN b.slots * f.guestcost 
           ELSE b.slots * f.membercost 
       END) AS total_revenue
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;
"""

# Execute the query and load the result into a DataFrame
df_Q10 = pd.read_sql_query(query, engine)

# Display the DataFrame
print(df_Q10)


   facility_name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


In [11]:

# Define the query
query = """
SELECT m1.surname AS member_surname, m1.firstname AS member_firstname, 
       m2.surname AS recommended_by_surname, m2.firstname AS recommended_by_firstname
FROM Members AS m1
LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m1.firstname;
"""

# Execute the query and load the result into a DataFrame
df_Q11 = pd.read_sql_query(query, engine)

# Display the DataFrame
print(df_Q11)


       member_surname member_firstname recommended_by_surname  \
0               Bader         Florence               Stibbons   
1               Baker             Anne               Stibbons   
2               Baker          Timothy                Farrell   
3              Boothe              Tim                 Rownam   
4             Butters           Gerald                  Smith   
5              Coplin             Joan                  Baker   
6             Crumpet            Erica                  Smith   
7                Dare            Nancy               Joplette   
8             Farrell            David                   None   
9             Farrell           Jemima                   None   
10              GUEST            GUEST                   None   
11            Genting          Matthew                Butters   
12               Hunt             John                Purview   
13              Jones            David               Joplette   
14              Jones    

In [12]:
# Define the query
query = """
SELECT f.name AS facility_name,
       SUM(b.slots) AS total_usage_by_members
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name
ORDER BY total_usage_by_members DESC;
"""

# Execute the query and load the result into a DataFrame
df_Q12 = pd.read_sql_query(query, engine)

# Display the DataFrame
print(df_Q12)


     facility_name  total_usage_by_members
0  Badminton Court                    1086
1   Tennis Court 1                     957
2   Massage Room 1                     884
3   Tennis Court 2                     882
4    Snooker Table                     860
5       Pool Table                     856
6     Table Tennis                     794
7     Squash Court                     418
8   Massage Room 2                      54


In [13]:
# Define the query
query = """
SELECT f.name AS facility_name,
       strftime('%Y-%m', b.starttime) AS month,
       SUM(b.slots) AS total_usage_by_members
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name, month
ORDER BY f.name, month;
"""

# Execute the query and load the result into a DataFrame
df_Q13 = pd.read_sql_query(query, engine)

# Display the DataFrame
print(df_Q13)


      facility_name    month  total_usage_by_members
0   Badminton Court  2012-07                     165
1   Badminton Court  2012-08                     414
2   Badminton Court  2012-09                     507
3    Massage Room 1  2012-07                     166
4    Massage Room 1  2012-08                     316
5    Massage Room 1  2012-09                     402
6    Massage Room 2  2012-07                       8
7    Massage Room 2  2012-08                      18
8    Massage Room 2  2012-09                      28
9        Pool Table  2012-07                     110
10       Pool Table  2012-08                     303
11       Pool Table  2012-09                     443
12    Snooker Table  2012-07                     140
13    Snooker Table  2012-08                     316
14    Snooker Table  2012-09                     404
15     Squash Court  2012-07                      50
16     Squash Court  2012-08                     184
17     Squash Court  2012-09                  